In [1]:
'''
This file prepares the raw transcriptomic data for Limma. 
Metadata is removed from raw file and moved to a metadata file.
Contrasts are created between low/control and low/high at each timepoint.
These 3 files can be processed with the Limma R Script or uploaded to Galaxy
'''

In [1]:
import pandas as pd
import numpy as np 

In [2]:
raw_counts = pd.read_csv('rna_vst_proc.csv')

/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_75956/430702266.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_counts = pd.read_csv('rna_vst_proc.csv')


In [40]:
#remove first column
raw_counts_dropped = raw_counts.iloc[:,1:]

#get column names prior to .# minus first column
column_names = raw_counts_dropped.columns
column_names_dropped = column_names[1:]
processed_column_names = [col.split('.')[0] for col in column_names_dropped]

#add column names to metadata file under condition
metadata_rna = pd.DataFrame(columns=['Sample','conditions','timepoints','conditions_timepoints'])
metadata_rna['conditions'] = processed_column_names

#get entries from time_point row minus first entry
timepoints_list = raw_counts_dropped.iloc[0, 1:].tolist()

#add to metadata file under timepoint
metadata_rna['timepoints'] = timepoints_list
metadata_rna['conditions_timepoints'] = metadata_rna['conditions'] + '_' + metadata_rna['timepoints']

#make list of Sample# - 1 - length of column
new_column_names = ['GeneID'] + [f'Sample{i+1}' for i in range(0, len(raw_counts_dropped.columns)-1)]
raw_counts_dropped.columns = new_column_names

#add column in metadata file with these column names
metadata_rna['Sample'] = new_column_names[1:]
#try with just one column
metadata_rna_cleaned = metadata_rna[['Sample','conditions_timepoints']]

#remove first two row from read counts
raw_counts_cleaned = raw_counts_dropped.iloc[2:,:]

,Sample,conditions_timepoints
0,Sample1,control_1H
1,Sample2,control_1H
2,Sample3,control_1H
3,Sample4,control_1H
4,Sample5,control_1H
5,Sample6,control_1H
6,Sample7,low_1H
7,Sample8,low_1H
8,Sample9,low_1H
9,Sample10,low_1H


In [47]:
#create contrasts file
#Define the conditions and time points
conditions = ['control', 'low', 'high']
timepoints = ['1H', '2H', '6H', '12H', '24H', '4D', '5D', '6D', '7D']

#Generate the contrasts
contrasts = []
for timepoint in timepoints:
    contrasts.append(f'low_{timepoint}-control_{timepoint}')
    contrasts.append(f'high_{timepoint}-control_{timepoint}')

#Create a DataFrame from the contrasts list
contrasts_df = pd.DataFrame(contrasts, columns=['Contrasts'])

,Contrasts
0,low_1H-control_1H
1,high_1H-control_1H
2,low_2H-control_2H
3,high_2H-control_2H
4,low_6H-control_6H
5,high_6H-control_6H
6,low_12H-control_12H
7,high_12H-control_12H
8,low_24H-control_24H
9,high_24H-control_24H


In [48]:
#save metadata and expression levels to csv files
raw_counts_cleaned.to_csv('raw_counts_cleaned.tsv',sep='\t',index=False)
metadata_rna_cleaned.to_csv('metadata_rna_cleaned.tsv',sep='\t',index=False)

#Save contrasts to a CSV file
contrasts_df.to_csv('contrasts.tsv', index=False, sep='\t')